In [ ]:
import scipy


all_theta = []
all_acc = []
for system, theta in sorted(list(zip(systems, trainer.last_params['ability'])), key=lambda v: v[1]):
    acc = np.average([
        line["score"][system] for line in data
    ])
    print(
        f"{system:>20}",
        f"Real acc: {acc:6.3f}",
        f"Inferred theta: {theta:6.3f}",
        sep=" | ",
    )
    all_theta.append(theta)
    all_acc.append(acc)

(scipy.stats.pearsonr(all_theta, all_acc)[0], scipy.stats.spearmanr(all_theta, all_acc)[0])

In [ ]:
def model_2pl(ability, diff, disc):
    return 1/(1+np.exp(-disc*(ability-diff)))


for system in systems:
    system_idx = trainer._dataset.subject_id_to_ix[system]
    item_idx = trainer._dataset.item_id_to_ix["item_10"]
    # trainer.irt_model.predict([system_idx], range(len(data)))
    y_true = trainer.irt_model.predict([system_idx], [item_idx])
    y_pred = model_2pl(
        ability=trainer.last_params["ability"][system_idx],
        diff=trainer.last_params["diff"][item_idx],
        disc=trainer.last_params["disc"][item_idx],
    )
    print(np.abs(y_true - y_pred))

In [ ]:
import matplotlib.pyplot as plt
import irt_mt_dev.utils.fig as figutils
figutils.matplotlib_default()

plt.figure(figsize=(4, 2))

min_ability = min(trainer.last_params["ability"])
max_ability = max(trainer.last_params["ability"])

print(min_ability, max_ability)

out_x = []
out_y = []
for ability in np.linspace(min_ability-0.1, max_ability+0.1, 100):
    preds = [
        model_2pl(ability=ability, diff=diff, disc=disc)
        for diff, disc in zip(trainer.last_params["diff"], trainer.last_params["disc"])
    ]
    out_x.append(ability)
    out_y.append(np.average(preds))

plt.plot(out_x, out_y)

out_x = []
out_y = []
for system in systems:
    system_idx = trainer._dataset.subject_id_to_ix[system]
    ability = trainer.last_params["ability"][system_idx]
    preds = [
        model_2pl(ability=ability, diff=diff, disc=disc)
        for diff, disc in zip(trainer.last_params["diff"], trainer.last_params["disc"])
    ]
    out_x.append(trainer.last_params["ability"][system_idx])
    out_y.append(np.average([line["metrics"][system]["MetricX-23-c"] >= 0.7 for line in data]))

plt.scatter(out_x, out_y, color="black")
plt.xlabel(r"System $\theta$")
plt.ylabel("Predicted success")
plt.show()